In [236]:
import nest_asyncio
import os
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

nest_asyncio.apply()

In [238]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from langchain_groq import ChatGroq
#Settings.llms=ChatGroq(model="llama3-8b-8192")
Settings.llm=Groq(model="llama3-70b-8192")
# embedding model
Settings.embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


In [239]:
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader


if 'index_store_dev' not in os.listdir():

    documents = SimpleDirectoryReader("./hack4", recursive = True).load_data()

    # chunk_size of 1024 is a good default value
    splitter = SentenceSplitter(chunk_size=1024)
    # Create nodes from documents
    nodes = splitter.get_nodes_from_documents(documents)
    
    # vector store index
    vector_index = VectorStoreIndex(nodes)

    os.makedirs("./index_store_dev/vector", exist_ok=True)


    vector_index.storage_context.persist(persist_dir="./index_store_dev/vector")

else:
    # rebuild storage context
    vector_storage_context = StorageContext.from_defaults(persist_dir="./index_store_dev/vector")

    # load index
    vector_index = load_index_from_storage(vector_storage_context)

In [240]:

# vector query engine
vector_query_engine = vector_index.as_query_engine()

In [241]:

'''
        prompt_template = """Text: {context}

        Question: {question}

        Answer the question based on the PDF Document provided. If the text doesn't contain the answer, reply that the answer is not available.
        Do Not Hallucinate"""


        PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "query"]
        )'''

'\n        prompt_template = """Text: {context}\n\n        Question: {question}\n\n        Answer the question based on the PDF Document provided. If the text doesn\'t contain the answer, reply that the answer is not available.\n        Do Not Hallucinate"""\n\n\n        PROMPT = PromptTemplate(\n            template=prompt_template, input_variables=["context", "query"]\n        )'

In [242]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [243]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Tamil Nadu query engine",
            description=(
                "Provides information about Tamil Nadu's services in great detail and mentions all the schemes etc available for the user "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )]

In [244]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")
'''agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    
)'''

'agent = ReActAgent.from_tools(\n    query_engine_tools,\n    llm=llm,\n    verbose=True,\n    \n)'

In [245]:
from llama_index.agent.openai import OpenAIAgent
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

In [25]:
from langchain_community.retrievers import TavilySearchAPIRetriever
import os
os.environ["TAVILY_API_KEY"] = "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"
retriever = TavilySearchAPIRetriever(k=3)
query = "women policy "

result=retriever.invoke(f'only from official goverment website of tamil nadu for the query:{query}')
source=[document.metadata['source'] for document in result]
for s in source:
    print(s)


https://www.tnsocialwelfare.tn.gov.in/en
https://spc.tn.gov.in/policy/state-policy-for-women-2021/
https://www.newindianexpress.com/states/tamil-nadu/2024/Feb/22/tn-govts-new-policy-lays-roadmap-for-womens-welfare-for-next-10-years


In [253]:
query=input("Enter the query:")
response=agent.chat(f'All details regarding the query asked in tamil nadu only and answer only what is asked no need to requestion ur own self, dont respond anything that is not from the documents. the query is : {query}')


Added user message to memory: All details regarding the query asked in tamil nadu only and answer it in great detail, dont respond anything that is not from the documents. the query is : What are the services provided by Tamil Nadu in the rural areas? List them.
=== Calling Function ===
Calling function: Tamil Nadu query engine with args: {"input":"What are the services provided by Tamil Nadu in the rural areas? List them."}
Got output: Based on the provided context, the services provided by Tamil Nadu in the rural areas include:

1. Primary Health Services
2. Electricity
3. Drinking Water
4. Toilets (although the availability is quite low)
5. Healthcare facilities, including:
   a. Community Health Centers (CHCs)
   b. Primary Health Centers (PHCs)
   c. Sub-Centers (SCs)

These services are mentioned in the context as part of the discussion on scaling up primary health services in rural Tamil Nadu.



In [254]:
print(str(response))

According to the information provided, the services provided by Tamil Nadu in the rural areas include:

1. Primary Health Services
2. Electricity
3. Drinking Water
4. Toilets (although the availability is quite low)
5. Healthcare facilities, including:
   a. Community Health Centers (CHCs)
   b. Primary Health Centers (PHCs)
   c. Sub-Centers (SCs)

These services are mentioned in the context as part of the discussion on scaling up primary health services in rural Tamil Nadu.
